# ios上架有一堆解析度需要調整，這裡做個工具幫忙調整

## MAC FFMPEG 安裝方式
```
brew install ffmpeg
```
https://juejin.cn/post/6844904191433900040
https://stackoverflow.com/questions/54161418/python3-6-add-audio-to-cv2-processed-video

6.5 吋 886 x 1920 像素（直向）  
5.5 吋  1080 x 1920   
4.7 吋   750 x 1334 像素  
12.9 吋   1200 x 1600   
9.7 吋    900 x 1200   

## 函式庫先執行

In [3]:
#opencv內建的Resize 比例會跑掉，這是用裁切的方式等比例放大或是縮小
def AkingResize(img , OutPutWidth , OutPutHeight):
    InimgW = img.shape[1]
    InimgH = img.shape[0]
    InRatio = InimgW / InimgH
    OutRatio = OutPutWidth / OutPutHeight
    if (OutRatio > InRatio): #更扁平 所以縮放的時候寬度一樣，原圖的高度要裁切
        #print ("壓扁")
        NewHeight = int(InimgW * OutPutHeight / OutPutWidth)
        CenterY = InimgH / 2
        StartY = int(CenterY - NewHeight / 2)
        EndY = StartY + NewHeight
        NewImg = img[StartY:EndY,0:InimgW]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    elif (OutRatio < InRatio): #更高長 所以縮放的時候高度一樣，原圖的寬度要裁切
        #print ("長高")
        NewWidth = int(InimgH * OutPutWidth / OutPutHeight)
        CenterX = InimgW / 2
        StartX = int(CenterX - NewWidth / 2)
        EndX = StartX + NewWidth
        NewImg = img[0:InimgH,StartX:EndX]
        Resizedimage = cv2.resize(NewImg, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    else: #相同比例
        Resizedimage = cv2.resize(img, (OutPutWidth, OutPutHeight), interpolation=cv2.INTER_CUBIC)
    return Resizedimage

## 照片調整

In [20]:
import cv2
InputPicPathName = r'/Users/aking/Desktop/IMG_6626.PNG'  #輸入影像的路徑與檔名
OutPutWidth = 720             #輸出寬度
OutPutHeight = 1280                #輸出高度
OutputPicPathName = r'/Users/aking/Desktop/IMG_720p.PNG'   #輸出影像的路徑與檔名
#=============開始轉換====================================
img = cv2.imread(InputPicPathName)
if (img.any() != None):
    Resizedimage = AkingResize(img , OutPutWidth , OutPutHeight)
    cv2.imwrite(OutputPicPathName, Resizedimage)
    # 顯示圖片
    #cv2.imshow('Resized image', Resizedimage)
    #cv2.waitKey(3)
    #cv2.destroyAllWindows()
    print ("圖片轉換完成")
    # 按下任意鍵則關閉所有視窗
else:
    print ("圖片讀取失敗")

圖片轉換完成


## 影片調整

In [11]:
import cv2
import os

InputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov'  #輸入影像的路徑與檔名
OutPutWidth = 1080            #輸出寬度
OutPutHeight = 1920                 #輸出高度
OutputVideoPathName = r'/Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava1080S.mp4'   #輸出影像的路徑與檔名


#=======================輸入設定====================================
video = cv2.VideoCapture(InputVideoPathName)
# 取得影像的尺寸大小
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = video.get(cv2.CAP_PROP_FPS) #每秒鐘幾張
print ("原始寬度{} 高度{} FPS{}".format(width , height,fps))


#=======================輸出檔案設定=======================
fourcc = cv2.VideoWriter_fourcc(*'avc1')
VideoFilePath = r"{}/TmpVideo.mp4".format(os.path.dirname(InputVideoPathName)) #解析度調整後的畫面
SoundFilePath = r"{}/sound.mp3".format(os.path.dirname(InputVideoPathName))    #原始檔案分離出聲音
VideoWithSoundFilePath = r"{}/TmpVideoWithSound.mp4".format(os.path.dirname(InputVideoPathName)) #畫面與聲音合成影片
#畫面與聲音合成影片 之後需要再轉檔一次 app store才會接受

out = cv2.VideoWriter(VideoFilePath, fourcc, fps, (int(OutPutWidth),  int(OutPutHeight)))
#=============開始轉換====================================
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    image = AkingResize(frame, OutPutWidth, OutPutHeight)
    out.write(image)
    cv2.imshow('frame', image)
    if cv2.waitKey(1) == ord('q'):
        break
video.release()
out.release()

CMD = r"ffmpeg -i {} -f mp3 -ab 256000 -vn {}".format(InputVideoPathName,SoundFilePath) #原始檔案分離出聲音
os.system(CMD) #原始檔案分離出聲音


CMD1 = r"ffmpeg -i {} -i {} -c copy -map 0:v:0 -map 1:a:0 {}".format(VideoFilePath, SoundFilePath , VideoWithSoundFilePath)
os.system(CMD1) #畫面與聲音合成影片

CMD2 = "ffmpeg -i {} -filter_complex 'scale={}:{},framerate=30' {}".format(VideoWithSoundFilePath , OutPutWidth,OutPutHeight,OutputVideoPathName)
os.system(CMD2) #最後再轉檔案一次 App store才會吃



os.system("rm -f {}".format(VideoFilePath)) 
os.system("rm -f {}".format(SoundFilePath)) 
os.system("rm -f {}".format(VideoWithSoundFilePath)) 



print("轉換完成")
cv2.destroyAllWindows()

原始寬度498.0 高度1080.0 FPS30.0


ffmpeg version 5.0 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.0.0 (clang-1300.0.29.30)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/5.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  lib

轉換完成


frame=  836 fps= 21 q=-1.0 Lsize=   33310kB time=00:00:27.90 bitrate=9779.0kbits/s speed=0.697x    
video:32929kB audio:350kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.093525%
[libx264 @ 0x7fd9dd706c00] frame I:15    Avg QP:21.40  size: 70132
[libx264 @ 0x7fd9dd706c00] frame P:248   Avg QP:24.47  size: 49710
[libx264 @ 0x7fd9dd706c00] frame B:573   Avg QP:25.75  size: 35495
[libx264 @ 0x7fd9dd706c00] consecutive B-frames:  6.5%  5.0%  4.3% 84.2%
[libx264 @ 0x7fd9dd706c00] mb I  I16..4: 19.2% 73.2%  7.6%
[libx264 @ 0x7fd9dd706c00] mb P  I16..4: 12.6% 40.5%  3.1%  P16..4: 20.5%  8.7%  2.4%  0.0%  0.0%    skip:12.1%
[libx264 @ 0x7fd9dd706c00] mb B  I16..4:  4.2% 10.1%  1.0%  B16..8: 33.2% 12.8%  2.5%  direct: 7.7%  skip:28.5%  L0:46.8% L1:46.5% BI: 6.8%
[libx264 @ 0x7fd9dd706c00] 8x8 transform intra:69.9% inter:81.5%
[libx264 @ 0x7fd9dd706c00] coded y,uvDC,uvAC intra: 41.9% 67.6% 28.4% inter: 27.0% 41.5% 0.8%
[libx264 @ 0x7fd9dd706c00] i16 v,h,dc,p: 48% 24% 11% 

# 懷舊程式碼

In [26]:
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "scale=1200:1600,framerate=30;"  /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout.mp4
ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava.mov -filter_complex "[0:v]setpts=1*PTS,scale=1200:1600,framerate=30[v];[0:a]atempo=8.0[a]" -map "[v]" -map "[a]" /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/ffout1600.mp4

ffmpeg -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Tmp.mp4 -i /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/sound.aac -c copy -map 0:v:0 -map 1:a:0 /Users/aking/Documents/2021Unity/LAVA/上架檔案/影片/Lava886S.mp4
